Whitney Kenner
UID: u0777962
5/31/23
CS6017
HW2

PART 1:

Null hypothesis:
A == B

the null hypothesis we want to disprove is that they are statistically the same....

In [ ]:
import math
#computing the Z value
p1 = 500/1000
p2 = 550/1000
p = (500 +550)/ (1000 + 1000)
n1 = 1000
n2 = 1000

zVal = (p1 - p2)/math.sqrt(p*(1-p)*(1/n1 + 1/n2))
print(zVal)

In [ ]:
#associated P value
import scipy.stats as sc
pVal = sc.norm.cdf(zVal)*100
print(pVal)

What can you conclude?
with a 1% significance level, we cannot make any conclusions about our null hypothesis
with a 5% significance level, we can disregard our null hypothesis since the p value is 1.25% or 0.0125, we can conclude that they are statistically not the same!!

PART 2:

Task 1:

In [ ]:
#import the csv files
import pandas as pd
realEstate1 = pd.read_csv('realEstate1.csv')
realEstate2 = pd.read_csv('realEstate2.csv')
frames = [realEstate1, realEstate2]
realEstate = pd.concat(frames)
realEstate


Task 2: clean the data

In [ ]:
#Only keep houses with List Price between 200,000 and 1,000,000 dollars
realEstate = realEstate.loc[realEstate['LstPrice'] <= 1000000]
realEstate = realEstate.loc[realEstate['LstPrice'] >= 200000]

#Remove columns that you don't think contribute to the value of the house
realEstate = realEstate.filter(items=['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio', 'PkgSpacs', 'PropType', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt'])
realEstate


In [ ]:
#Convert 'TotSqf' to an integer.
realEstate['TotSqf']=realEstate['TotSqf'].str.replace(',','')
realEstate['TotSqf'] = realEstate['TotSqf'].astype(int)
realEstate


In [ ]:
#Add a new column titled Prop_Type_SingleFamily that is set to 0 if the i-th listing is a condo or townhouse, or 1 if the i-th listing is a single family house.
realEstate['Prop_Type_SingleFamily'] = [1 if x == 'Single Family' else 0 for x in realEstate['PropType']]
realEstate

In [ ]:
#Remove the listings with erroneous 'Longitude' (one has Longitude = 0) and 'Taxes' values (two have unreasonably large values).
realEstate = realEstate.loc[(realEstate['Taxes'] < 10000)]
realEstate = realEstate.loc[(realEstate['Longitude'] != 0)]
#realEstate.sort_values(['Longitude'])
#realEstate.sort_values(['Taxes'])



Task 3: Exploratory Data Analysis

Explore the dataset. Write a short description of the dataset describing the number of items, the number of variables and check to see if the values are reasonable.

This dataset contains 260 rows (items) and 16 columns (variables) 

Unreasonable values:
Most values seem reasonable except:
- 1 house has 54 patios which seems absurd
- 1 person is spending $1 on taxes
- 1 house has 0 bathrooms which is presumably illegal

everything else seems quite reasonable and this data can be evaluated to look at correlations between these values and house price

In [ ]:
#Make a bar chart showing the breakdown of the different types of houses (single family, townhouse, condo).
import matplotlib.pyplot as plt
plt.style.use( 'ggplot' )
realEstate['PropType'].value_counts().plot(kind="bar")



In [ ]:
#Compute the correlation matrix and use a heat map to visualize the correlation coefficients.
#Use a diverging color scale from -1 to +1 (see vmin and vmax parameters for pcolorLinks to an external site.)
#Show a legend (colorbarLinks to an external site.)
#Make sure the proper labels are visible and readable (see xticksLinks to an external site. and the corresponding yticksLinks to an external site.).
import seaborn as sn
modifiedHMDF = realEstate.filter(items=['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio', 'PkgSpacs', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt'])
corrHMDF = modifiedHMDF.corr()
realEstateHeatMap = sn.heatmap(corrHMDF, vmax=1, vmin=-1)

In [ ]:
#Make a scatter plot matrix to visualize the correlations. Determine which columns have strong correlations.
scatterREDF = realEstate.filter(items=['Acres', 'LstPrice', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt'])
pd.plotting.scatter_matrix(scatterREDF, figsize=(12,12))

The columns with strong correlation are: 
sold price and lst price, which intuitively makes sense that the sale price would be close to the listed price. This has the strongest correlation, but some weaker ones (but still positively correlated) are:
Total square feet and taxes
list price and total square feet
sold price and total square feet

these are somewhere between a moderate and strong correlation. A strong correlation is generally around an r of 0.7, which is what I would predict the last strong relationships are

outside of the relationships listed above, there seems to be little to no correlation between the various variables of a house

year built, in particular, seems to be the weakest indicater of anything at all as there is no correlation between it and anything else

Task 4: Geospatial Plot

Two of the variables are the latitude and longitude of each listing. Salt Lake City is on this nice east-west, north south grid, so even a simple plot of latitude vs longitude makes sense. Create a scatterplot of these two variables. Use color to indicate the price of the house. How does the price depend on the house location?

Bonus: If you can, overlay the scatterplot on a map of the city. (This is challenging, and we didn't teach you how to do it, so you should do the other parts of the assignment first.)

What can you say about the relation between the location and the house price?

In [ ]:
#plot the latitude vs longitude on a scatterplot (use color to indicate the price of the house) How does the price of the house depend on location?
import numpy as np
locationDF = realEstate.filter(items=['Latitude', 'Longitude', 'LstPrice'])


locationScatterPlot = plt.scatter(x=locationDF['Latitude'], y=locationDF['Longitude'], c=locationDF['LstPrice'])
cbar = plt.colorbar(locationScatterPlot)
cbar.ax.set_yticklabels(['200,000', '300,000','400,000','500,000', '600,000','700,000', '800,000','900,000', '1,000,000'])
cbar.set_label("List Price of House")
plt.xlabel("Latitude")
plt.ylabel("Longitude")




How does the price depend on the house location?
the farther east and north you go, (within this collection of locations), the price generally increases.

What can you say about the relation between the location and the house price?
Most of the houses range between 200k and 700k, but as you get farther and farther east and north, the most expensive houses (between 700k and 1 million), are found on the outer edges. It would seem in the more central areas, location is not a very important facter for price, but for the eastern and northern houses, the location is correlated with higher house price!

Task 5: Simple Linear Regression

In [ ]:
import statsmodels.formula.api as sm
#sold price vs taxes ols
sold_taxes_ols = sm.ols(formula="SoldPrice ~ Taxes", data=realEstate).fit()
sold_taxes_ols.summary()

In [ ]:
#sold price vs acres ols
sold_acres_ols = sm.ols(formula="SoldPrice ~ Acres", data=realEstate).fit()
sold_acres_ols.summary()

In [ ]:
#list price vs sold price ols
sold_list_ols = sm.ols(formula="SoldPrice ~ LstPrice", data=realEstate).fit()
sold_list_ols.summary()

You'll find that the best predictor of sold price is the list price. Report the R-squared value for this model (SoldPrice ~ LstPrice) and give an interpretation for its meaning. 

The R-squared value for (SoldPrice ~ LstPrice) is 0.990, which is a very good R-squared value. This model explains 99% of the variablility in SldPrice! The closer an R-squared value is to 1, the greater the data is correlated, so this value indicates there is an extremely strong correlation between sale and list price.


Also give an interpretation of beta_1 for this model.

Beta_1 is the slope of the line, and in this case that's 1.0488, which means that as the listed price goes up, the sale price also goes up slightly more than 1. This will be reflected in the graph below. This slope indicates that the sale price on average increases linearly with the list price!


In [ ]:
# Make a plot of list price vs. sold price and overlay the prediction coming from your regression model.
plt.scatter(x=realEstate['LstPrice'], y=realEstate['SoldPrice'], c='k', marker="*", label='LstPrice')
m= 1.0488
b= -9277.1587
plt.plot(realEstate['LstPrice'], m*realEstate['LstPrice'] + b, color="blue", linewidth=3)
_ =plt.xlabel("List Price ($)")
_ =plt.ylabel("Sold Price ($)")

Task 6: Multilinear Regression

In [ ]:
#Develop a multilinear regression model for house prices in this neighborhood.
real_estate_all_ols = sm.ols(formula="SoldPrice ~ Acres + Deck + GaragCap + Latitude + Longitude + Patio + PkgSpacs + Taxes + TotBed + TotBth + TotSqf + YearBlt", data=realEstate).fit()
real_estate_all_ols.summary()


Which variables are the best predictors for the Sold Price?

The best predictors for the sold price are: Taxes and TotSqf. There are other values with high coefficients but also high p values which indicate the model between that variable and sold price are not significant

Specific questions:

- Often the price per square foot for a house is advertised. Is this what the coefficient for TotSqf is measuring? Provide an interpretation for the coefficient for TotSqf.

The coefficient for TotSqf is the slope of the line for how sold price increases based on total square feet increasing. In this example, the average house price across this set will increase by 41.40 for every 1 ft squared increase in size in the house

- Estimate the value that each Garage space adds to a house.

$23,810. BUT the correlation between SalePrice and GaragCap is only 63%, so this is not a particularly strong correlation, so this is a rough estimate based on moderatley correlated values

- Does latitude or longitude have an impact on house price? Explain.

latitude does not. It has an extremely high p-value, so it can not be considered stastically significant. 
Longitude has a p value of 0.001 which is considered statistically significant but there is only a correlation of 39% which is very weak

- If we wanted to start a 'house flipping' company, we'd have to be able to do a better job of predicting the sold price than the list price does. How does your model compare?

It's not great, our R-squared value is quite a bit worse and many of these variables have weak or no correlation, we would definitely want a better model if we wanted our company to be succesful 

In [ ]:
realEstateCoefficients = realEstate.filter(items=['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio', 'PkgSpacs', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt'])
print(realEstateCoefficients.corr())

Task 7: Incorporating a Categorical Variable

In [ ]:
#sold price vs proptype
sold_prop_ols = sm.ols(formula="SoldPrice ~ Prop_Type_SingleFamily", data=realEstate).fit()
sold_prop_ols.summary()


In [ ]:
# sold price vs proptype & total square feet
sold_prop_sqft_ols = sm.ols(formula="SoldPrice ~ Prop_Type_SingleFamily + TotSqf", data=realEstate).fit()
sold_prop_sqft_ols.summary()


From the first model, it would appear that Property type is significant in predicting the sold price. On the other hand, the second model indicates that when you take into account total square footage, property type is no longer predictive. Explain this. (Hint: there is a confounder lurking here.) 

- Total square feet is a cofounder for the sold price and property type. When the property type is compared on it's own to sold price it has a p value of 0.000 BUT it's R-squared value is horrendous (0.156). When adding in the total square feet to this model, the r value becomes 0.741, which is substantially better, and the p value increases for the house type to a point where it is no longer statistically significant

In [ ]:
#Make a scatterplot of TotSqf vs. SoldPrice where the house types are colored differently to illustrate your explanation
sold_sqf_prop = realEstate.filter(items=['TotSqf', 'PropType', 'SoldPrice'])
colors = {'Single Family':'red', 'Townhouse': 'green', 'Condo':'blue'}
sold_sqf_prop_plt = plt.scatter(x=sold_sqf_prop['TotSqf'], y=sold_sqf_prop['SoldPrice'], c=sold_sqf_prop['PropType'].map(colors))

plt.xlabel("Total Sqare Feet")
plt.ylabel("Sold Price")
